In [1]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [2]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release-20160523/bin'
#PROGRAM_PATHS = '/scratch/BS/release-BSR/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
#PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = '/scratch/SuperResolution/BRAINSSuperResolution/HCPWorkflows/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

In [3]:
DWI_scan = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs/DWI_corrected_alignedSpace.nrrd'
EdgeMap_scan = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs/EdgeMap.nrrd'

In [4]:
sessionID = 105115
WFname = 'HCPWorkflow_SR_' + str(sessionID)

In [7]:
if PYTHON_AUX_PATHS is not None:
    Path_to_Matlab_Func = '/scratch/SuperResolution/BRAINSSuperResolution/HCPWorkflows/'
    assert os.path.exists(Path_to_Matlab_Func), "Path to SR matlab function is not found: %s" % Path_to_Matlab_Func

###### UTILITY FUNCTIONS #######
#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/#
# Run Super resolution reconstruction by Matlab
def runSRbyMatlab(inputDWI,inputEdgeMap,Path_to_Matlab_Func):
    import os
    import nipype.interfaces.matlab as matlab
    script="runSR('"+inputDWI+"','"+inputEdgeMap+"','"+os.getcwd()+"')"
    mlab = matlab.MatlabCommand()
    mlab.set_default_matlab_cmd("matlab")
    mlab.inputs.single_comp_thread = False
    mlab.inputs.nodesktop = True
    mlab.inputs.nosplash = True
    mlab.inputs.paths = Path_to_Matlab_Func
    mlab.inputs.script = script
    mlab.run()
    baseline_fn = os.path.join(os.getcwd(), 'DWI_Baseline.nrrd')
    assert os.path.isfile(baseline_fn), "DWI baseline file is not found: %s" % baseline_fn
    NN_fn = os.path.join(os.getcwd(), 'DWI_NN.nrrd')
    assert os.path.isfile(NN_fn), "DWI Nearest Neighbor file is not found: %s" % NN_fn
    IFFT_fn = os.path.join(os.getcwd(), 'DWI_IFFT.nrrd')
    assert os.path.isfile(IFFT_fn), "DWI IFFT file is not found: %s" % IFFT_fn
    TV_fn = os.path.join(os.getcwd(), 'DWI_TV.nrrd')
    assert os.path.isfile(TV_fn), "DWI TV file is not found: %s" % TV_fn
    WTV_fn = os.path.join(os.getcwd(), 'DWI_WTV.nrrd')
    assert os.path.isfile(WTV_fn), "DWI WTV file is not found: %s" % WTV_fn   
    return [baseline_fn,NN_fn,IFFT_fn,TV_fn,WTV_fn]

#################################
#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\#

SRWF = pe.Workflow(name=WFname)

BASE_DIR = os.path.join('/scratch/TESTS/IpythonNotebook/20160615_HCPWF', '2_SRWF')
SRWF.base_dir = BASE_DIR

inputsSpec = pe.Node(interface=IdentityInterface(fields=['DWIVolume',
                                                         'EdgeMap']),
                     name='inputsSpec')

inputsSpec.inputs.DWIVolume = DWI_scan
inputsSpec.inputs.EdgeMap = EdgeMap_scan

outputsSpec = pe.Node(interface=IdentityInterface(fields=['DWI_baseline',
                                                          'DWI_NN',
                                                          'DWI_IFFT',
                                                          'DWI_TV',
                                                          'DWI_WTV']),
                      name='outputsSpec')

runSR=pe.Node(interface=Function(function = runSRbyMatlab,
                                 input_names=['inputDWI','inputEdgeMap','Path_to_Matlab_Func'],
                                 output_names=['baseline_fn','NN_fn','IFFT_fn','TV_fn','WTV_fn']),
              name="runSR")
runSR.inputs.Path_to_Matlab_Func = Path_to_Matlab_Func
SRWF.connect([(inputsSpec,runSR,[('DWIVolume','inputDWI'),('EdgeMap','inputEdgeMap')])])

SRWF.connect([(runSR,outputsSpec,[('baseline_fn','DWI_baseline'),
                                  ('NN_fn','DWI_NN'),
                                  ('IFFT_fn','DWI_IFFT'),
                                  ('TV_fn','DWI_TV'),
                                  ('WTV_fn','DWI_WTV')])])


SRWF.write_graph()
SRWF.run()

/IPLlinux/raid0/homes/aghayoor/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [ ]:
'''
###### UTILITY FUNCTIONS #######
#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/#
# Normalize DWI scan components between zero and one
def NormalizeDWIComponents(inputDWI):
    import os
    import numpy as np
    from ReadWriteNrrdDWI import ReadNAMICDWIFromNrrd, WriteNAMICDWIToNrrd
    # utility function
    def _3DNormalizeBetweenZeroAndOne(arr):   
        newMax = 1.0
        newMin = 0.0
        oldMax = float(np.amax(arr))
        oldMin = float(np.amin(arr))
        f=(newMax-newMin)/(oldMax-oldMin)
        normalized_arr = (arr-oldMin)*f+newMin
        return normalized_arr
    #
    assert os.path.exists(inputDWI), "File not found: %s" % inputDWI
    dwi_data,dwi_header,bvecs,bvals,gradient_index = ReadNAMICDWIFromNrrd(inputDWI)
    dwi_data = dwi_data.astype(np.float32)
    numOfComponents = dwi_data.shape[gradient_index]
    for idx in range(numOfComponents):
        dwi_3d = dwi_data[idx,:,:,:]
        dwi_3d_norm = _3DNormalizeBetweenZeroAndOne(dwi_3d)
        dwi_data[idx,:,:,:] = dwi_3d_norm
    outNormalizedDWIFilename = os.path.join(os.getcwd(), 'DWI_baseline.nrrd')
    # write corrected nrrd file to disk
    WriteNAMICDWIToNrrd(outNormalizedDWIFilename,dwi_data,bvecs,bvals,dwi_header)
    assert os.path.isfile(outNormalizedDWIFilename), "Normalized DWI file is not found: %s" % outNormalizedDWIFilename
    return outNormalizedDWIFilename
        
def CreateMatFiles(inputDWI,inputEdgemap):
    import scipy.io
    import SimpleITK as sitk
    from ReadWriteNrrdDWI import ReadNAMICDWIFromNrrd
    #
    assert os.path.exists(inputDWI), "File not found: %s" % inputDWI
    assert os.path.exists(inputEdgemap), "File not found: %s" % inputEdgemap
    #
    edgemap = sitk.ReadImage(inputEdgemap)
    edge_arr = sitk.GetArrayFromImage(edgemap)
    edgemap_mat_file = os.path.join(os.getcwd(),'edgemap.mat')
    scipy.io.savemat(edgemap_mat_file, mdict={'edgemap': edge_arr})
    #
    dwi_data,dwi_header,bvecs,bvals,gradient_index = ReadNAMICDWIFromNrrd(inputDWI)
    # ...
    #
    return [edgemap_mat_file, dwi_mat_file]
#################################
#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\#

SRWF = pe.Workflow(name=WFname)

BASE_DIR = os.path.join('/scratch/TESTS/IpythonNotebook/20160615_HCPWF', '2_SRWF')
SRWF.base_dir = BASE_DIR

inputsSpec = pe.Node(interface=IdentityInterface(fields=['DWIVolume',
                                                         'EdgeMap']),
                     name='inputsSpec')

inputsSpec.inputs.DWIVolume = DWI_scan
inputsSpec.inputs.EdgeMap = EdgeMap_scan

outputsSpec = pe.Node(interface=IdentityInterface(fields=['DWI_baseline'
                                                          ,'DWI_NN'
                                                          ,'DWI_IFFT'
                                                          ,'DWI_TV'
                                                          ,'DWI_WTV'
                                                         ]),
                      name='outputsSpec')

##
## STEP 1: Create DWI_baseline that is normalized version of input corrected aligned DWI volume
##
CreateDWIBaseline = pe.Node(interface=Function(function = NormalizeDWIComponents,
                                                        input_names=['inputDWI'],
                                                        output_names=['outNormalizedDWIFilename']),
                                     name="CreateDWIBaseline")
SRWF.connect(inputsSpec, 'DWIVolume', CreateDWIBaseline, 'inputDWI')
SRWF.connect(CreateDWIBaseline, 'outNormalizedDWIFilename', outputsSpec, 'DWI_baseline')


##
## STEP 2: Convert input images to matlab arrays
##
CreateMatFilesNode = pe.Node(interface=Function(function = CreateMatFiles,
                                                      input_names=['inputDWI','inputEdgemap'],
                                                      output_names=['edgemap_mat_file','dwi_mat_file']),
                             name="CreateMatFiles")
SRWF.connect(inputsSpec, 'DWIVolume', CreateMatFilesNode, 'inputDWI')
SRWF.connect(inputsSpec, 'EdgeMap', CreateMatFilesNode, 'inputEdgemap')
#


SRWF.write_graph()
SRWF.run()
'''